In [1]:
from mfusampler.main import uni_slice
import numpy as np
from scipy.stats import multivariate_normal

def _MfU_fEval(xk, k, x, fMulti, **kwargs):
    x[k] = xk
    return fMulti(x, **kwargs)

def _scalar_or_length_n(x, n):
    if np.isscalar(x):
        return np.repeat(x, n)
    elif len(x) == n:
        return x
    else:
        raise ValueError("Input must be of length 1 or 'n'")

def MfU_control(
    n
    , slice_w = 1.0, slice_m = np.inf
    , slice_lower = -np.inf, slice_upper = +np.inf
):
    slice_w = _scalar_or_length_n(slice_w, n)
    slice_m = _scalar_or_length_n(slice_m, n)
    slice_lower = _scalar_or_length_n(slice_lower, n)
    slice_upper = _scalar_or_length_n(slice_upper, n)
    ret = {'slice': {'w': slice_w, 'm': slice_m, 'lower': slice_lower, 'upper': slice_upper}}
    return ret

def MfU_sample(x, f, uni_sampler = 'slice', uni_sampler_control = None, **kwargs):
    N = len(x)
    if uni_sampler_control is None:
        uni_sampler_control = MfU_control(N)
    control_slice = uni_sampler_control['slice']
    for n in range(N):
        if uni_sampler == 'slice':
            x[n] = uni_slice(
                x[n], f = _MfU_fEval, k = n, x = x, fMulti = f
                , w = control_slice['w'][n]
                , m = control_slice['m'][n]
                , lower = control_slice['lower'][n]
                , upper = control_slice['upper'][n]
                , **kwargs
            )
        else:
            raise ValueError('Invalid univariate sampler')
    return x

def MfU_sample_run(x, f, uni_sampler = 'slice', uni_sampler_control = None, nsmp = 10, **kwargs):
    xall = np.empty([nsmp, len(x)])
    for n in range(nsmp):
        x = MfU_sample(x, f, uni_sampler, uni_sampler_control, **kwargs)
        xall[n, :] = x
    return xall

In [2]:
x_init = np.array([0.0, 0.0])
my_mean = np.array([1.0, -1.0])
my_cov = np.array([[1.0, 0.0], [0.0, 1.0]])

def my_multi(x, mean, cov):
    return np.log(multivariate_normal.pdf(x, mean, cov))

_MfU_fEval(+0.5, 0, x_init, my_multi, mean = my_mean, cov = my_cov)

-2.4628770664093453

In [3]:
MfU_control(2)

{'slice': {'w': array([1., 1.]),
  'm': array([inf, inf]),
  'lower': array([-inf, -inf]),
  'upper': array([inf, inf])}}

In [4]:
MfU_sample(x = x_init, f = my_multi, mean = my_mean, cov = my_cov)

array([ 0.91699845, -0.57331112])

In [28]:
nsmp = int(1e3)
import time

t = time.time()
xsmp = MfU_sample_run(x = x_init, f = my_multi, mean = my_mean, cov = my_cov, nsmp = nsmp)
t = time.time() - t
print(f'elapsed time: {round(t, 1)}sec')

elapsed time: 2.71sec


In [25]:
np.mean(xsmp[(nsmp//2):, :], axis = 0), my_mean

(array([ 1.07448362, -1.12247415]), array([ 1., -1.]))

In [24]:
np.cov(xsmp[(nsmp//2):, :].transpose()), my_cov

(array([[ 0.97492063, -0.02002185],
        [-0.02002185,  1.08088712]]),
 array([[1., 0.],
        [0., 1.]]))